## Assignment 22.1
### ACD MDS (Mar 2018 batch) Student: K. Anandaranga

In [2]:
# Problem statement: 
# Treat this as a classification problem by creating a new binary variable affair (did the woman have at least one affair?) 
# and trying to predict the classification for each woman

In [4]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from patsy import dmatrices

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

In [5]:
dta = sm.datasets.fair.load_pandas().data

In [6]:
# add "affair" column: 1 represents having affairs, 0 represents not

dta['affair'] = (dta.affairs > 0).astype(int)

In [7]:
y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + religious + educ + C(occupation) + C(occupation_husb)', dta, return_type="dataframe")

In [10]:
X = X.rename(columns = {'C(occupation)[T.2.0]':'occ_2',
                        'C(occupation)[T.3.0]':'occ_3',
                        'C(occupation)[T.4.0]':'occ_4',
                        'C(occupation)[T.5.0]':'occ_5',
                        'C(occupation)[T.6.0]':'occ_6',
                        'C(occupation_husb)[T.2.0]':'occ_husb_2',
                        'C(occupation_husb)[T.3.0]':'occ_husb_3',
                        'C(occupation_husb)[T.4.0]':'occ_husb_4',
                        'C(occupation_husb)[T.5.0]':'occ_husb_5',
                        'C(occupation_husb)[T.6.0]':'occ_husb_6'})


In [15]:
# Flatten y to a 1D array

y = np.ravel(y)

In [16]:
# Run Logistic Regression
model = LogisticRegression()
model = model.fit(X,y)

In [17]:
model.score (X,y)

0.72588752748978946

In [18]:
# Model prediction accuracy is 72.3%

In [19]:
# Co-efficients
pd.DataFrame(list(zip(X.columns, np.transpose(model.coef_))))

,0,1
0,Intercept,[1.48983589132]
1,occ_2,[0.188066390244]
2,occ_3,[0.498947866816]
3,occ_4,[0.250668564985]
4,occ_5,[0.839008064812]
5,occ_6,[0.833908433744]
6,occ_husb_2,[0.190635944587]
7,occ_husb_3,[0.297832712926]
8,occ_husb_4,[0.161408854076]
9,occ_husb_5,[0.18777091389]


In [20]:
# Coefficients are negative for:
#  rate_marriage
#  religious
#  So, probability of having an affair decreases if the marriage rating is high and similarly
#  if the religiousness is high

# Age is also having a negative coeff... so, probability of having an affair
#  decreases as the women age

In [22]:
# Now, spliting into train and test
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size = 0.3, random_state = 0)

model2 = LogisticRegression()
model2.fit (X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
# predict class labels for the test set
predicted = model2.predict(X_test)
print (predicted)

[ 1.  0.  0. ...,  0.  0.  0.]


In [24]:
# Generate class probabilities
probs = model2.predict_proba(X_test)
print (probs)

[[ 0.3514634   0.6485366 ]
 [ 0.90955084  0.09044916]
 [ 0.72567333  0.27432667]
 ..., 
 [ 0.55727385  0.44272615]
 [ 0.81207043  0.18792957]
 [ 0.74734601  0.25265399]]


In [25]:
# From the above it is clear that 50% is the threshold for classification

In [26]:
# generate evaluation metrics
print (metrics.accuracy_score(y_test, predicted))
print (metrics.roc_auc_score(y_test, probs[:,1]))

0.729842931937
0.745950606951


In [27]:
print (metrics.confusion_matrix(y_test, predicted))
print (metrics.classification_report(y_test,predicted))

[[1169  134]
 [ 382  225]]
             precision    recall  f1-score   support

        0.0       0.75      0.90      0.82      1303
        1.0       0.63      0.37      0.47       607

avg / total       0.71      0.73      0.71      1910



In [28]:
#Evaluate the model using 10-fold cross-validation

scores = cross_val_score(LogisticRegression(),X, y, scoring = 'accuracy', cv = 10)
print (scores)
print (scores.mean())

[ 0.72100313  0.70219436  0.73824451  0.70597484  0.70597484  0.72955975
  0.7327044   0.70440252  0.75157233  0.75      ]
0.724163068551


In [ ]:
# Final result: Logistic Regression model is consistently predicting at 72% accuracy